In [1]:

%pprint

Pretty printing has been turned OFF


In [2]:

import sys

# Insert at 1, 0 is the script path (or '' in REPL)
sys.path.insert(1, '../py')

In [3]:

# Get the Neo4j driver
from storage import Storage
s = Storage()

from ha_utils import HeaderAnalysis
ha = HeaderAnalysis(verbose=False)

from scrape_utils import WebScrapingUtilities
wsu = WebScrapingUtilities(s=s)
uri = wsu.secrets_json['neo4j']['connect_url']
user =  wsu.secrets_json['neo4j']['username']
password = wsu.secrets_json['neo4j']['password']

from cypher_utils import CypherUtilities
cu = CypherUtilities(uri=uri, user=user, password=password, driver=None, s=s, ha=ha)

In [4]:

from neo4j.exceptions import ServiceUnavailable

try:
    version_str = cu.driver.verify_connectivity()
    print(f'======== {version_str} ========')
    
    from hc_utils import HeaderCategories
    hc = HeaderCategories(cu=cu, verbose=False)
    
    from section_utils import SectionUtilities
    su = SectionUtilities(s=s, ha=ha, cu=cu, verbose=False)
    
    from lr_utils import LrUtilities
    lru = LrUtilities(ha=ha, cu=cu, hc=hc, verbose=False)
    
    from crf_utils import CrfUtilities
    crf = CrfUtilities(ha=ha, hc=hc, cu=cu, verbose=False)
    
    import warnings
    warnings.filterwarnings('ignore')
except ServiceUnavailable as e:
    # print(str(e).strip())
    raise ServiceUnavailable('You need to start Neo4j as a console')
except Exception as e:
    print(e.__class__)

======== Neo4j/4.4.7 ========


In [5]:

import time
import humanize
from pandas import DataFrame
import os
from datetime import datetime
print(f'Last run on {datetime.now()}')

Last run on 2022-08-16 13:24:37.087696



---
# Training

In [6]:

t0 = time.time()
lru.build_pos_logistic_regression_elements(verbose=False)
crf.retrain_pos_classifier(verbose=False)
duration_str = humanize.precisedelta(time.time() - t0, minimum_unit='seconds', format='%0.0f')
print(f'Parts-of-speech classifier retrained in {duration_str}')

Parts-of-speech classifier retrained in 8 minutes and 46 seconds


In [7]:

t0 = time.time()
lru.build_isheader_logistic_regression_elements(verbose=False)
lru.retrain_isheader_classifier(verbose=False)
duration_str = humanize.precisedelta(time.time() - t0, minimum_unit='seconds', format='%0.0f')
print(f'Is-header classifier retrained in {duration_str}')

Is-header classifier retrained in 6 seconds


In [8]:

# Rebuild the classifer from the quals dictionary
t0 = time.time()
lru.build_isqualified_logistic_regression_elements(verbose=False)
lru.retrain_isqualified_classifier(verbose=False)
duration_str = humanize.precisedelta(time.time() - t0, minimum_unit='seconds', format='%0.0f')
print(f'Is-qualified classifer retrained in {duration_str}')

Is-qualified classifer retrained in 11 seconds


In [9]:

navigable_parent_cypher_str = '''
    MATCH (np:NavigableParents {{navigable_parent: '{}'}})
    ''' + cu.return_everything_str + ';'
cypher_str = f'''
    MATCH (fn:FileNames)
    WHERE
        fn.percent_fit < 0.4 AND
        ((fn.is_closed IS NULL) OR (fn.is_closed = false)) AND
        ((fn.is_verified IS NULL) OR (fn.is_verified = false)) AND
        ((fn.is_opportunity_application_emailed IS NULL) OR
        (fn.is_opportunity_application_emailed = false))
    RETURN
        fn.percent_fit AS percent_fit,
        fn.file_name AS file_name,
        fn.posting_url AS url
    ORDER BY fn.percent_fit ASC;'''
row_objs_list = []
with cu.driver.session() as session:
    row_objs_list = session.write_transaction(cu.do_cypher_tx, cypher_str)
files_list = []
if row_objs_list:
    files_list = DataFrame(row_objs_list).file_name.tolist()
print(f'Only {len(files_list)} more mis-estimated minimum-requirements-met percentages to go!')

Only 80 more mis-estimated minimum-requirements-met percentages to go!



----

In [10]:

# file_name = files_list.pop()
file_name = 'e89d4437aa0bbd56_Bind_Senior_Data_Scientist_Telecommute_Minneapolis_MN_55421_Indeed_com.html'
file_path = os.path.join(cu.SAVES_HTML_FOLDER, file_name)
page_soup = wsu.get_page_soup(file_path)
div_soup = page_soup.find_all(name='div', id='jobDescriptionText')[0]
child_strs_list = ha.get_navigable_children(div_soup, [])
cu.ensure_filename(file_name, verbose=False)
cu.populate_from_child_strings(child_strs_list, file_name, verbose=False)
print(file_name)

e89d4437aa0bbd56_Bind_Senior_Data_Scientist_Telecommute_Minneapolis_MN_55421_Indeed_com.html


In [11]:

child_tags_list = ha.get_child_tags_list(child_strs_list)
is_header_list = []
for is_header, child_str in zip(ha.get_is_header_list(child_strs_list), child_strs_list):
    if is_header is None:
        probs_list = lru.ISHEADER_PREDICT_PERCENT_FIT(child_str)
        idx = probs_list.index(max(probs_list))
        is_header = [True, False][idx]
    is_header_list.append(is_header)
feature_dict_list = hc.get_feature_dict_list(child_tags_list, is_header_list, child_strs_list)
feature_tuple_list = []
for feature_dict in feature_dict_list:
    feature_tuple_list.append(hc.get_feature_tuple(feature_dict, lru.pos_lr_predict_single))
crf_list = crf.CRF.predict_single(crf.sent2features(feature_tuple_list))
db_pos_list = []
for navigable_parent in child_strs_list:
    db_pos_list = cu.append_parts_of_speech_list(navigable_parent, pos_list=db_pos_list)
pos_list, indices_list = su.visualize_basic_quals_section(crf_list, child_strs_list, db_pos_list=db_pos_list, verbose=True)

['O-CS', 'O-SP', 'O-CS', 'O-CS', 'O-TS', 'H-TS', 'O-TS', 'O-TS', 'O-TS', 'O-TS', 'O-TS', 'O-TS', 'O-TS', 'O-TS', 'O-TS', 'H-RQ', 'O-ER', 'O-PQ', 'O-RQ', 'O-RQ', 'O-RQ', 'O-RQ', 'O-RQ', 'O-TS', 'H-PQ', 'O-ER', 'O-RQ', 'O-RQ', 'O-RQ', 'O-LN', 'O-CS', 'O-CS', 'O-SP', 'O-LN', 'O-SP', 'O-TS', 'O-LN', 'O-LN']
[16, 18, 19, 20, 21, 22]


[16, 18, 19, 20, 21, 22]


In [ ]:
raise

In [45]:

# Display the context of an individual child string
idx = 29
print(indices_list); child_str = child_strs_list[idx]; pos_symbol = pos_list[idx]; basic_quals_dict = s.load_object('basic_quals_dict'); print(str(basic_quals_dict[child_str]) + '\n' if(child_str in basic_quals_dict) else '', end=''); print(f'{idx} {pos_symbol}) {child_str}')

[16, 18, 19, 20, 21, 22]
29 O-LN) To protect the health and safety of our workforce, patients and communities we serve, UnitedHealth Group and its affiliate companies require all employees to disclose COVID-19 vaccination status prior to beginning employment. In addition, some roles and locations require full COVID-19 vaccination, including boosters, as an essential job function. UnitedHealth Group adheres to all federal, state and local COVID-19 vaccination regulations as well as all client COVID-19 vaccination requirements and will obtain the necessary information from candidates prior to employment to ensure compliance. Candidates must be able to perform all essential job functions with or without reasonable accommodation. Failure to meet the vaccination requirement may result in rescission of an employment offer or termination of employment


In [43]:

# Hand-label this particular child string in the quals dictionary
basic_quals_dict = s.load_object('basic_quals_dict')
basic_quals_dict[child_str] = 1
s.store_objects(basic_quals_dict=basic_quals_dict); print(f'"{child_str}" in basic_quals_dict: {basic_quals_dict[child_str]}')

Pickling to C:\Users\daveb\OneDrive\Documents\GitHub\job-hunting\saves\pkl\basic_quals_dict.pkl
"Familiarity with Agile or other rapid application development methods" in basic_quals_dict: 1


In [44]:

def do_cypher_tx(tx, navigable_parent, verbose=False):
    cypher_str = """MATCH (np:NavigableParents {navigable_parent: $navigable_parent})
        SET
            np.is_header = 'False',
            np.is_task_scope = 'False',
            np.is_minimum_qualification = 'False',
            np.is_preferred_qualification = 'True',
            np.is_educational_requirement = 'False',
            np.is_legal_notification = 'False',
            np.is_other = 'False',
            np.is_corporate_scope = 'False',
            np.is_job_title = 'False',
            np.is_office_location = 'False',
            np.is_job_duration = 'False',
            np.is_supplemental_pay = 'False',
            np.is_interview_procedure = 'False',
            np.is_posting_date = 'False'
        """ + cu.return_everything_str + ';'
    results_list = tx.run(query=cypher_str, parameters={'navigable_parent': navigable_parent})

    return [dict(record.items()) for record in results_list]
with cu.driver.session() as session:
    row_objs_list = session.write_transaction(do_cypher_tx, navigable_parent=child_str)
row_objs_list

[{'navigable_parent': 'Familiarity with Agile or other rapid application development methods', 'is_header': 'False', 'is_task_scope': 'False', 'is_minimum_qualification': 'False', 'is_preferred_qualification': 'True', 'is_legal_notification': 'False', 'is_job_title': 'False', 'is_office_location': 'False', 'is_job_duration': 'False', 'is_supplemental_pay': 'False', 'is_educational_requirement': 'False', 'is_interview_procedure': 'False', 'is_corporate_scope': 'False', 'is_posting_date': 'False', 'is_other': 'False'}]

In [ ]:

# Show what's in the database already for this html string
# print(navigable_parent_cypher_str.format(child_str))
with cu.driver.session() as session:
    row_objs_list = session.write_transaction(cu.do_cypher_tx, navigable_parent_cypher_str.format(child_str))
row_objs_list


----

In [46]:

# Mark the file name as needing retraining everywhere
import numpy as np
from IPython.display import clear_output

mask_series = lru.hunting_df.percent_fit.isin([file_name])
lru.hunting_df.loc[mask_series, 'percent_fit'] = np.nan
s.store_objects(hunting_df=lru.hunting_df)
def do_cypher_tx(tx, file_name, verbose=False):
    cypher_str = """
        MATCH (fn:FileNames {file_name: $file_name})
        SET fn.percent_fit = NULL, fn.is_verified = false
        RETURN fn;"""
    if verbose:
        clear_output(wait=True)
        print(cypher_str.replace('$file_name', f'"{file_name}"'))
    results_list = tx.run(query=cypher_str, parameters={'file_name': file_name})

    return [dict(record.items()) for record in results_list]
with cu.driver.session() as session:
    row_objs_list = session.write_transaction(do_cypher_tx, file_name=file_name, verbose=False)
row_objs_list

Pickling to C:\Users\daveb\OneDrive\Documents\GitHub\job-hunting\saves\pkl\hunting_df.pkl


[{'fn': <Node id=854396 labels=frozenset({'FileNames'}) properties={'file_name': 'e89d4437aa0bbd56_Bind_Senior_Data_Scientist_Telecommute_Minneapolis_MN_55421_Indeed_com.html', 'posting_url': 'https://www.indeed.com/rc/clk/dl?jk=e89d4437aa0bbd56&from=ja&qd=RnZhMybXSk4M3QtTVGXWoZj-R_bxcYib5xeGNtZ7GZa7JuDVyNzHCTIYZZigs6bd2Uv-mplXiphsCiQp6vpp7MmM6tiD7btQXxFBJCIPMss&rd=PHewGpBsnC2hNEVS_u8nVmW4xXBGTIWlVfOl08776OE&tk=1gaeu6e4phuj8800&alid=6254377b33b425113af40aa2', 'is_verified': False}>}]

In [ ]:

# You've made no changes to the qualification dictionary because it looks good as is
from IPython.display import clear_output

def do_cypher_tx(tx, file_name, verbose=False):
    cypher_str = """
        MATCH (fn:FileNames {file_name: $file_name})
        SET fn.is_verified = true
        RETURN fn;"""
    if verbose:
        clear_output(wait=True)
        print(cypher_str.replace('$file_name', f'"{file_name}"'))
    parameter_dict = {'file_name': file_name}
    results_list = tx.run(query=cypher_str, parameters=parameter_dict)
    values_list = []
    for record in results_list:
        values_list.append(dict(record.items()))

    return values_list
with cu.driver.session() as session:
    row_objs_list = session.write_transaction(do_cypher_tx, file_name=file_name, verbose=True)
row_objs_list

In [ ]:

# Remove this particular child string from the quals dictionary
basic_quals_dict = s.load_object('basic_quals_dict')
child_str = child_strs_list[idx]
basic_quals_dict.pop(child_str)
# basic_quals_dict[child_str] = 0
s.store_objects(basic_quals_dict=basic_quals_dict)
print(f'"{child_str}" in basic_quals_dict: {child_str in basic_quals_dict}')

In [ ]:

# Mark the file name as closed
cypher_str = f'''
    MATCH (fn:FileNames {{file_name: "{file_name}"}})
    SET fn.is_closed = true
    RETURN fn;'''
print(cypher_str)
with cu.driver.session() as session:
    row_objs_list = session.write_transaction(cu.do_cypher_tx, cypher_str)
row_objs_list

In [ ]:

# Manually label the unscored qual
qualification_str = quals_list[13]
print(qualification_str)
basic_quals_dict = s.load_object('basic_quals_dict')
basic_quals_dict[qualification_str] = 0
s.store_objects(basic_quals_dict=basic_quals_dict)

In [ ]:

prediction_list = list(lru.predict_job_hunt_percent_fit(quals_list))
basic_quals_dict = s.load_object('basic_quals_dict')
lru.basic_quals_dict = basic_quals_dict
quals_str, qual_count = lru.get_quals_str(prediction_list, quals_list)
job_fitness = qual_count/len(prediction_list)
job_title = file_name.replace('.html', '').replace('_', ' ')
display(HTML(f'<p>I only meet {job_fitness:.1%} of the minimum requirements for the {job_title} position, but I can explain:</p>'))
for i, qual_str in enumerate(quals_list):
    if qual_str in basic_quals_dict:
        if basic_quals_dict[qual_str]:
            idx = qual_str.find('>')
            if idx == -1:
                display(HTML(f'{i+1}) {qual_str}'))
            else:
                display(HTML(f'{qual_str[:idx+1]}{i+1}) {qual_str[idx+1:]}'))


----

In [ ]:

basic_quals_dict = s.load_object('basic_quals_dict')
for key in basic_quals_dict.keys():
    if 'automat' in key.lower():
        print(key)

In [ ]:

# Mark the files with the largest qualification (implying it was run together) as needing to be retrained
import numpy as np

hunting_df = s.load_object('hunting_df')
basic_quals_dict = s.load_object('basic_quals_dict')
child_str = sorted([child_str for child_str in basic_quals_dict.keys()], key=lambda x: len(x), reverse=True)[0]
print(child_str)

In [ ]:

# Mark the files with the largest qualification (implying it was run together) as needing to be retrained
basic_quals_dict.pop(child_str)
s.store_objects(basic_quals_dict=basic_quals_dict)
def do_cypher_tx(tx, navigable_parent, verbose=False):
    cypher_str = """
        MATCH (np:NavigableParents {navigable_parent: $navigable_parent})-[r:NEXT]->(:NavigableParents)
        RETURN r.file_name AS file_name;"""
    if verbose:
        clear_output(wait=True)
        print(cypher_str.replace('$navigable_parent', f'"{navigable_parent}"'))
    tx.run(cypher_str, navigable_parent=navigable_parent)
with cu.driver.session() as session:
    row_objs_list = session.write_transaction(do_cypher_tx, navigable_parent=child_str, verbose=True)
for row_obj in row_objs_list:
    file_name = row_obj['file_name']
    mask_series = hunting_df.file_name.isin([file_name])
    hunting_df.loc[mask_series, 'percent_fit'] = np.nan
    s.store_objects(hunting_df=hunting_df)
    def do_cypher_tx(tx, file_name, verbose=False):
        cypher_str = """
        MATCH (fn:FileNames {file_name: $file_name})
        SET fn.percent_fit = NULL;"""
        if verbose:
            clear_output(wait=True)
            print(cypher_str.replace('$file_name', f'"{file_name}"'))
        tx.run(cypher_str, file_name=file_name)
    with cu.driver.session() as session:
        session.write_transaction(do_cypher_tx, file_name=file_name, verbose=False)

In [ ]:

# Find a qual in the dictionary with this substring
sentence_regex = re.compile(r'[\.;]')
quals_set = set()
concatonated_quals_list = sentence_regex.split(child_str.replace('<div>', '').replace('</div>', '').strip())
for q in concatonated_quals_list:
    q = q.strip()
    if q:
        quals_set.add(q)
quals_list = list(quals_set)
for q in quals_list:
    print(q)

In [ ]:

prediction_list

In [ ]:

print(quals_str)

In [ ]:

qual_count